## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/Users/hani/Documents/llm-zoomcamp/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
user_question = "I just discovered the course. Can I still join it?"
vector = embedding_model.encode(user_question)
print("The first value of the embeded question: ", round(vector[0], 3))

The first value of the embeded question:  0.078


## Prepare the documents

In [11]:
# Load the documents with ids that we prepared in the module:

import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [18]:
ml_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(ml_documents)

375

In [20]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

## Q2. Creating the embeddings

In [21]:
# we will create an embedding for both question and answer fields.
# We want to put all of them into a single matrix `X`:

embeddings = []

for doc in ml_documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))

import numpy as np
X = np.array(embeddings)
X.shape



(375, 768)

## Q3. Search

In [31]:
# let's compute the cosine similarity between the vector from Q1 
# and the matrix from Q2. 
scores = X.dot(vector)
print("Highest Score of Cosine Similarity Scores: ", round(scores.max(), 3))

Highest Score of Cosine Similarity Scores:  0.651
